This function enables patterns generated via Ernould's method to be written to a .up2 file to enable processing by ATEX 

In [1]:
import numpy as np
import os
import sys

# Add the parent directory to the Python path
sys.path.append(os.path.abspath("/Users/crestiennedechaine/Scripts/pyHREBSD/"))

import ErnouldsMethod
import write_up2
import conversions

In [2]:
Ernould_PC = np.array([600, 600, 16*1000/20]) #the pattern center to be used for Ernould's method
detector_shape = (1200,1200)

In [3]:
#EMEBSD pattern numbers 

#EMpatnum = np.array([2, 6, 11, 21, 26, 36, 46, 51, 56, 60, 64, 69, 79, 84, 94, 104, 109, 114, 118, 122, 127, 137, 142, 152, 162, 167, 172, 176, 180, 185, 195, 200, 210, 220, 225, 230, 234, 238, 243, 253, 258, 268, 278, 283, 288, 292, 296, 301, 311, 316, 326, 336, 341, 346])
#subtract 1 to convert to zero indexing
#EMpatnum_zeroindex = EMpatnum - 1





In [4]:
# number = 2 
# EMfile = f'/Users/crestiennedechaine/Scripts/pyHREBSD/EMEBSDpats_Nov52025/EBSD_Al_Nov52025_Pat_{number}.h5'

# EMEBSDpatterns = np.zeros((len(EMpatnum), 1200, 1200), dtype=np.uint16)
# for i, number in enumerate(EMpatnum):
#     number = EMpatnum[i]
#     EMfile = f'/Users/crestiennedechaine/Scripts/pyHREBSD/EMEBSDpats_Nov52025/EBSD_Al_Nov52025_Pat_{number}.h5'
#     print(f"Processing pattern number: {number} from file: {EMfile}")
#     pat = ErnouldsMethod.read_EMEBSD(EMfile)
#     EMEBSDpatterns[i] = pat

In [5]:
EMEBSDfilename = '/Users/crestiennedechaine/Scripts/pyHREBSD/EBSDpattern_Al_July222025.h5'
#Testcasesfilename =  '/Users/crestiennedechaine/Scripts/pyHREBSD/results/StrainAndRotation-Aug42025.csv'
Testcasesfilename = '/Users/crestiennedechaine/Scripts/pyHREBSD/Ernoulds Method/Altering_e23.csv'

outputfile = '/Users/crestiennedechaine/Scripts/pyHREBSD/E23_Nov302025.up2'
EMEBSD_pattern = ErnouldsMethod.read_EMEBSD(EMEBSDfilename)

#determine the reference pattern coordinates need to pass in PC and detector shape
ref_coords = ErnouldsMethod.pattern_coords(Ernould_PC, detector_shape)
            #Read in the test cases from a csv file
            #load in the testcases
testcases_data = np.genfromtxt(Testcasesfilename, delimiter=',', encoding='utf-8-sig')
            #breaking the testcases data into euler angles and strain/rotation components
        
w = testcases_data[:, 0:3]
e = testcases_data[:, 3:]

#create the tensor Fe_values
Fe_values = np.zeros((w.shape[0], 3, 3), dtype=np.float32)
#calculate the Fe values for each test case
for i in range(w.shape[0]):
    Fe_values[i, :, :] = ErnouldsMethod.determineF(e[i], w[i])







In [6]:
#save the Fe values and w values to the same csv file
output_filename = '/Users/crestiennedechaine/Scripts/pyHREBSD/results/ErnouldsMethod_Fe_Values_smallstrainapprox_Nov232025.csv'
header = 'w1,w2,w3,Fe11,Fe12,Fe13,Fe21,Fe22,Fe23,Fe31,Fe32,Fe33'
data_to_save = np.hstack((w, Fe_values.reshape(w.shape[0], 9)))
np.savetxt(output_filename, data_to_save, delimiter=',', header=header, comments='', fmt='%.10e')

In [7]:

h = conversions.F2h(Fe_values, [0, 0, 16*1000/20])



In [8]:
#save the h values to a csv file
output_h_filename = '/Users/crestiennedechaine/Scripts/pyHREBSD/results/ErnouldsMethod_h_Values_Nov232025.csv'
header_h = 'h1,h2,h3,h4,h5,h6,h7,h8'
np.savetxt(output_h_filename, h.T, delimiter=',', header=header_h, comments='')



In [9]:
#example pattern 
F = np.eye(3)
example = ErnouldsMethod.generate_patterns(F, Ernould_PC, ref_coords, EMEBSD_pattern)

In [10]:
counter = 0 
n_patterns = w.shape[0]
generated_patterns = np.zeros((n_patterns, detector_shape[0], detector_shape[1]), dtype=np.float32)
for idx in range(n_patterns):
    # #if idx is equal to a number in EMpatnum_zeroindex, use the corresponding EMEBSD pattern
    # if idx in EMpatnum_zeroindex:
    #     generated_patterns[idx, :, :] = EMEBSDpatterns[counter, :, :]
    #     counter += 1
    # else:
    EMEBSD_pattern = ErnouldsMethod.read_EMEBSD(EMEBSDfilename)
    Fe = Fe_values[idx]
    #add the calculated Fe to a tensor 
    #determine the coordinate transformation
    generated_patterns[idx, :, :] = ErnouldsMethod.generate_patterns(Fe, Ernould_PC, ref_coords, EMEBSD_pattern)



In [11]:
#save every tenth pattern as a .png file to check
import matplotlib.pyplot as plt

#folder to save the images
output_folder = '/Users/crestiennedechaine/Scripts/pyHREBSD/GeneratedPatterns_Oct9/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for i in range(0, n_patterns, 10):
    plt.imsave(os.path.join(output_folder, f'pattern_{i:03d}.png'), generated_patterns[i], cmap='gray')
    

In [12]:

write_up2.write_up2(generated_patterns, outputfile, bit_depth=16)